Demonstration of quantEM config for gpus and such. 

Requirements: 
- This notebook requires an available GPU(s) or apple silicon MPS (MPS is largely untested but should work)

Arthur McCray
December 17, 2025

In [1]:
from quantem.core import config
import torch 

The most common use for `config` is setting and getting the default device. Laptops will have at maximum one gpu, or some apple devices have `mps`. In these cases it is rather simple to keep track of which device is being used, but for servers and workstations with multiple gpus it is important to specify which gpu to use.  

In [2]:
print(f"Torch cuda is available: {torch.cuda.is_available()}")
print(f"Number of GPUs available: {torch.cuda.device_count()}")
print(f"GPU names: {[torch.cuda.get_device_name(i) for i in range(torch.cuda.device_count())]}")
print(f"Torch mps is available: {torch.backends.mps.is_available()}")

Torch cuda is available: True
Number of GPUs available: 4
GPU names: ['NVIDIA L40S', 'NVIDIA L40S', 'NVIDIA L40S', 'NVIDIA L40S']
Torch mps is available: False


We will proceed with selecting the first gpu (index 0)

In [3]:
print("default device: ", config.get("device"))  
config.set({"device":0})  # set device 
print("set device: ", config.get("device"))  # current device, torch string format
config.refresh()  # reset to defaults
print("device after refresh: ", config.get("device"))

default device:  cpu
set device:  cuda:0
device after refresh:  cpu


There are helpers for accessing the default compute device, as that's the most frequently used part of the config. 

You can set the default device by passing an integer (corresponding to the GPU index), a `torch.device` object, or a `torch`-style string, e.g. `"cuda:0"` to specify the first GPU.  

In [4]:
### ways to get the current device
print(f"Initial device: {config.get('device')} | {config.get_device()}")

### ways to set the device
### any of the following will work
config.set_device(0) # integer index of the gpu
config.set_device("cuda:0") # torch-style string
config.set_device(torch.device(0)) # torch device object
# config.set_device("mps") # if using an apple device with mps

print(f"device after setting: {config.get_device()}")

### reset to defaults
config.refresh()
print(f"device after refresh: {config.get_device()}")


Initial device: cpu | cpu
device after setting: cuda:0
device after refresh: cpu


The default device determines where a `torch` tensor will be sent if you set its device as `"cuda"`. However it is generally preferable to be explicit and send tensors to the named device with `.to(config.get_device)`. 

A couple of other notes: 
- Setting defaults or updating the config will throw an error if there isn't a gpu at the index specified 
- Setting the device will also specify the gpu used by `cupy` if it is in your environment.
    - `cupy` is no longer a dependency of quantEM, but it's part of why the config was written in the first place. 

In [5]:
print("starting quantem device: ", config.get_device())
t = torch.arange(5)
print("tensor created on: ", t.device)
t = t.to("cuda")
print("tensor moved to `cuda` lands on: ", t.device)

try:
    config.set({"device": 1})
except RuntimeError as e:
    print("Only 1 gpu available, so this fails")
    
print("quantem device set to: ", config.get_device(), " (torch tensors are still created on cpu unless otherwise specified)")
t = t.to("cuda")
print("tensor moved to `cuda` lands on: ", t.device)
config.refresh()

starting quantem device:  cpu
tensor created on:  cpu
tensor moved to `cuda` lands on:  cuda:0
quantem device set to:  cuda:1  (torch tensors are still created on cpu unless otherwise specified)
tensor moved to `cuda` lands on:  cuda:1


In [6]:
try:
    config.set_device(4)  ## throws an error if only 1 gpu
except RuntimeError as e:
    print(f"Only {torch.cuda.device_count()} gpus available, so this fails with:\n'''\n{e}'''\n")

Only 4 gpus available, so this fails with:
'''
CUDA error: invalid device ordinal
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
'''



You can change the defaults (for this session/kernel) with `config.update_defaults`, this will not be overwritten by `refresh`

In [7]:
print(f"starting defaults: device={config.get('device')} | dtype_real={config.get('dtype_real')}")  
config.update_defaults({"device": 0})  # setting gpu with the index also works
config.set({"dtype_real": "float64"})  
print(f"updating the default device to {config.get('device')} and changing dtype_real (without updating the default) to {config.get('dtype_real')}")  
print(f"before refresh: device={config.get('device')} | dtype_real={config.get('dtype_real')}")
config.refresh()  # reset to defaults
print(f"after refresh:  device={config.get('device')} | dtype_real={config.get('dtype_real')}")
config.update_defaults({"device": "cpu"})  # setting back to original defaults


starting defaults: device=cpu | dtype_real=float32
updating the default device to cuda:0 and changing dtype_real (without updating the default) to float64
before refresh: device=cuda:0 | dtype_real=float64
after refresh:  device=cuda:0 | dtype_real=float32


#### if you want to persist a config change, you can use the write method
Setting the defaults using `update_defaults` does not persist if you restart the kernel. It just updates the current defaults and therefore changes what is reset when calling `config.refresh()`. 

The defaults are initially defined on import when the nmodule looks for a user-specified config file. If one exists it will overwrite the quantem defaults with the user defaults. 

(The default location for the user file is "~/.config/quantem/config.yaml", in line with abtem which puts it at "~/.config/abtem/". The quantem defaults are stored in "quantem/core/quantem.yaml")

In [8]:
config.refresh()
print("device after refresh: ", config.get("device"))
config.set({"device": "cuda:0"})
config.write()
config.refresh()
print("device after refresh: ", config.get("device"))


device after refresh:  cpu
writing config to:  /home/amccray/.config/quantem/config.yaml
device after refresh:  cuda:0


At this point if you restart the kernel, the default device will still be "cuda:0"

In [1]:
from pathlib import Path
from quantem.core import config

print(config.device())

cuda:0


For this demo we of course don't want to actually set the config, so lets undo that by removing the config file we just made

In [2]:
## reset to defaults by removing the config file
configfile = Path("~/.config/quantem/config.yaml").expanduser()
configfile.unlink()
config.refresh()
print("device after deleting the defaults file and refresh: ", config.get("device"))

## we could also just reset the config to defaults by setting the device to "cpu"
## but this would leave the config file in place, so we'll delete it instead
# config.set_device("cpu")
# config.write()
# config.refresh()
# print("device after deleting the defaults file and refresh: ", config.get("device"))

device after deleting the defaults file and refresh:  cpu


## Other stuff in config

Internally, within the `config` module there is a `config.config` dictionary that keeps track of all settings (default or user specified). You can access this with `config.get` and `config.set`, or by directly looking at the `config.config` dictionary.  
Can revert to defaults with `refresh`

`quantEM` now has `torch` as a required dependency, and does not require `cupy`. This was not always the case, and you can therefore use the config to check if the environment has `cupy` and `torch`, in order to protect from import errors 

In [3]:
print("has cupy: ", config.get("has_cupy"))
print("has torch: ", config.get("has_torch"))

has cupy:  False
has torch:  True


You can see all the things specified in `config` by looking directly at the dictionary: 

In [4]:
config.config

{'has_torch': True,
 'has_cupy': False,
 'device': 'cpu',
 'precision': 'float32',
 'dtype_real': 'float32',
 'dtype_complex': 'complex64',
 'verbose': 1,
 'cupy': {'fft-cache-size': '0 MB'},
 'mkl': {'threads': 2},
 'warnings': {'suppress-all-': False},
 'viz': {'interpolation': 'nearest',
  'real_space_units': 'A',
  'reciprocal_space_units': 'A^-1',
  'cmap': 'gray',
  'phase_cmap': 'magma',
  'default_colors': '',
  'colors': {'set': ['#3A7D44',
    '#E83F85',
    '#775AEB',
    '#ED8607',
    '#74D4B5',
    '#808080',
    '#C51D20',
    '#7C6A0A',
    '#00B4D8',
    '#774936'],
   'paired': ['#3A7D44',
    '#A2C899',
    '#E83F85',
    '#FFA5C5',
    '#775AEB',
    '#C2B4F4',
    '#ED8607',
    '#F9C689',
    '#74D4B5',
    '#C2F0DE',
    '#808080',
    '#C0C0C0',
    '#C51D20',
    '#F28E8E',
    '#7C6A0A',
    '#C5B86A',
    '#00B4D8',
    '#80D9EB',
    '#774936',
    '#B38B7D']}}}

There are some vizualization defaults in config that can be changed. 

In [5]:
config.get('viz')

{'interpolation': 'nearest',
 'real_space_units': 'A',
 'reciprocal_space_units': 'A^-1',
 'cmap': 'gray',
 'phase_cmap': 'magma',
 'default_colors': '',
 'colors': {'set': ['#3A7D44',
   '#E83F85',
   '#775AEB',
   '#ED8607',
   '#74D4B5',
   '#808080',
   '#C51D20',
   '#7C6A0A',
   '#00B4D8',
   '#774936'],
  'paired': ['#3A7D44',
   '#A2C899',
   '#E83F85',
   '#FFA5C5',
   '#775AEB',
   '#C2B4F4',
   '#ED8607',
   '#F9C689',
   '#74D4B5',
   '#C2F0DE',
   '#808080',
   '#C0C0C0',
   '#C51D20',
   '#F28E8E',
   '#7C6A0A',
   '#C5B86A',
   '#00B4D8',
   '#80D9EB',
   '#774936',
   '#B38B7D']}}

-- end -- 